In [14]:
import numpy as np

from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb

In [15]:
def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))

In [16]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [17]:

# В этом коде происходит следующее:
# 
# Загружаются данные диабета с помощью load_diabetes() из sklearn.datasets.
# X - признаки (data), y - целевая переменная (target)
# Создается модель регрессии XGBRegressor с линейной регрессией как целевой функцией и заданным random_state.
# Обучаем модель на данных X и y.
# Делаем предсказание на тех же данных X.
# Вычисляем среднеквадратичную ошибку между предсказанными значениями и истинными y.
# Выводим корень из среднеквадратичной ошибки - то есть RMSE (root mean squared error).
# Таким образом, в коде обучается модель регрессии XGB на данных о диабете, делаются предсказания и оценивается их качество с помощью RMSE.
diabetes = load_diabetes()

X = diabetes.data
y = diabetes.target

xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)

xgb_model.fit(X, y)

y_pred = xgb_model.predict(X)

mse=mean_squared_error(y, y_pred)

print(np.sqrt(mse))

0.39512221537109155


/home/dastan/Code/BigData/venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:32:52] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [18]:
# Переменная xgb_model содержит обученную модель регрессии XGBRegressor.
xgb_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, objective='reg:linear', ...)

In [19]:

# В этом коде происходит следующее:
# 
# Загружаются данные о раке молочной железы с помощью load_breast_cancer() из sklearn.datasets.
# X - признаки, y - целевая переменная (0 или 1)
# Создается модель классификации XGBClassifier с логистической регрессией как целевой функцией и фиксированным random_state.
# Обучаем модель на данных X и y.
# Делаем предсказание на тех же данных X.
# Выводим матрицу ошибок (confusion matrix) по истинным значениям y и предсказанным y_pred.
# Матрица ошибок показывает, как часто модель:
# 
# Правильно предсказывала класс 0 (истинно отрицательные)
# Неправильно предсказывала класс 0 как 1 (ложно положительные)
# Правильно предсказывала класс 1 (истинно положительные)
# Неправильно предсказывала класс 1 как 0 (ложно отрицательные)
# Таким образом, мы обучили модель классификации и оценили качество её предсказаний с помощью матрицы ошибок.
cancer = load_breast_cancer()

X = cancer.data
y = cancer.target

xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X, y)

y_pred = xgb_model.predict(X)

print(confusion_matrix(y, y_pred))

[[212   0]
 [  0 357]]


In [20]:

# В этом примере происходит следующее:
# 
# Загружаются данные о винах (wine dataset) с помощью load_wine() из sklearn.datasets.
# X - признаки (характеристики вин), y - метки классов (сорта вин)
# Создается модель классификации XGBClassifier, где целевая функция "multi:softprob" для мультиклассовой классификации (3 класса вина).
# Модель обучается на данных X и y.
# Делаем предсказания на тех же данных.
# Выводим матрицу ошибок между предсказанными метками y_pred и истинными y.
# Матрица ошибок покажет, как часто модель:
# 
# Правильно предсказывала каждый класс
# Неправильно классифицировала объекты одного класса как другой
# Таким образом, мы обучили модель для мультиклассовой классификации вин и оценили качество с помощью матрицы ошибок.
wine = load_wine()

X = wine.data
y = wine.target

xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X, y)

y_pred = xgb_model.predict(X)

print(confusion_matrix(y, y_pred))

[[59  0  0]
 [ 0 71  0]
 [ 0  0 48]]


In [21]:

# В этом коде происходит:
# 
# Загрузка данных о диабете.
# Разбиение данных на обучающую и тестовую выборки с помощью кросс-валидации KFold.
# Обучение модели XGBRegressor на каждой обучающей выборке.
# Предсказание на соответствующей тестовой выборке.
# Вычисление ошибки предсказания в виде среднеквадратичной ошибки для каждого разбиения.
# Сохранение всех ошибок в список scores.
# Вывод среднеквадратичного корня ошибок как итогового результата работы модели.
# Таким образом, это реализация кросс-валидации для оценки качества модели регрессии на наборе данных о диабете. Кросс-валидация позволяет более объективно оценить способность модели обобщаться на незнакомых данных.
diabetes = load_diabetes()

X = diabetes.data
y = diabetes.target

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    xgb_model = xgb.XGBRegressor(objective="reg:linear")
    xgb_model.fit(X_train, y_train)

    y_pred = xgb_model.predict(X_test)

    scores.append(mean_squared_error(y_test, y_pred))

display_scores(np.sqrt(scores))

/home/dastan/Code/BigData/venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:32:53] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/dastan/Code/BigData/venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:32:54] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/dastan/Code/BigData/venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:32:54] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/dastan/Code/BigData/venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:32:54] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.

Scores: [57.8878367  58.38510758 68.69963127 65.94107893 64.05194017]
Mean: 62.993
Std: 4.235


/home/dastan/Code/BigData/venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:32:54] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [22]:
# В этом коде происходит следующее:
# 
# Создание модели XGBRegressor с линейной регрессией в качестве целевой функции.
# Применение кросс-валидации с помощью функции cross_val_score.
# Параметры:
# 
# Модель: xgb_model
# Данные: X и y
# Оценка качества: среднеквадратичная ошибка со знаком минус (neg_mean_squared_error)
# Количество разбиений: 5
# Функция cross_val_score возвращает массив из 5 значений ошибки для каждого разбиения.
# Преобразование ошибок в RMSE вычислением корня и сменой знака (поскольку изначально ошибки со знаком минус).
# Вывод полученных RMSE с помощью функции display_scores.
# Таким образом, этот код позволяет быстро реализовать кросс-валидацию с оценкой по RMSE для модели XGBRegressor, не писать явные циклы.
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)

scores = cross_val_score(xgb_model, X, y, scoring="neg_mean_squared_error", cv=5)

display_scores(np.sqrt(-scores))

/home/dastan/Code/BigData/venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:32:54] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/dastan/Code/BigData/venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:32:54] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/dastan/Code/BigData/venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:32:54] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/dastan/Code/BigData/venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:32:54] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.

Scores: [55.68547317 58.18498778 68.622534   64.15281814 68.4826639 ]
Mean: 63.026
Std: 5.286


In [23]:
# В этом коде происходит следующее:
# 
# Загружаются данные о диабете.
# Создается модель XGBRegressor без указания гиперпараметров.
# Определяются возможные значения гиперпараметров для поиска в виде словаря params. Для каждого параметра задается распределение случайных чисел (функции uniform и randint).
# Создается объект RandomizedSearchCV для случайного перебора комбинаций гиперпараметров. Указываются:
# модель xgb_model
# пространство поиска params
# количество итераций
# кросс-валидация с 3 разбиениями
# вывод прогресса поиска
# количество параллельных задач
# сохранение значений метрики для обучающей выборки
# Вызывается метод поиска fit.
# Из полученных результатов cv_results выводятся лучшие найденные комбинации гиперпараметров и значения метрики качества с помощью функции report_best_scores.
# Таким образом, этот код реализует случайный поиск гиперпараметров для модели XGBRegressor с оценкой качества по кросс-валидации.
cancer = load_breast_cancer()

X = cancer.data
y = cancer.target

# if more than one evaluation metric are given the last one is used for early stopping
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="auc")

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

xgb_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test)])

y_pred = xgb_model.predict(X_test)

accuracy_score(y_test, y_pred)

[0]	validation_0-auc:0.96286
[1]	validation_0-auc:0.97742
[2]	validation_0-auc:0.97514


/home/dastan/Code/BigData/venv/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[3]	validation_0-auc:0.98096
[4]	validation_0-auc:0.98637
[5]	validation_0-auc:0.99043
[6]	validation_0-auc:0.98918
[7]	validation_0-auc:0.98980
[8]	validation_0-auc:0.98939
[9]	validation_0-auc:0.99251
[10]	validation_0-auc:0.99209
[11]	validation_0-auc:0.99168
[12]	validation_0-auc:0.99313
[13]	validation_0-auc:0.99313
[14]	validation_0-auc:0.99355
[15]	validation_0-auc:0.99313
[16]	validation_0-auc:0.99272
[17]	validation_0-auc:0.99272
[18]	validation_0-auc:0.99293
[19]	validation_0-auc:0.99293


0.965034965034965

In [24]:
# Распечатывается лучшее значение метрики качества (best_score) и номер итерации, на котором оно было достигнуто (best_iteration), из обученной модели xgb_model.
# Эти параметры сохраняются в модели после обучения и позволяют посмотреть результаты валидации/кросс-валидации.
# 
# Таким образом, в коде вычисляется метрика качества на тесте и выводятся метрики по итогам обучения модели.
print("Лучший скор: {0}, Лучшая итерация: {1}".format(xgb_model.best_score, xgb_model.best_iteration))

Лучший скор: 0.9935497295047857, Лучшая итерация: 14


In [25]:

# В этом коде происходит следующее:
# 
# Загружаются данные о раке молочной железы.
# Данные разбиваются на обучающую и тестовую выборки с помощью train_test_split.
# Создается модель XGBClassifier для бинарной классификации, указываются параметры:
# objective - логистическая регрессия
# n_estimators - количество деревьев в ансамбле
# eval_metric - метрики качества для оценки на валидации: AUC, ошибка классификации и ошибка при пороге 0.6
# Модель обучается на обучающей выборке, при этом качество оценивается на тестовой выборке eval_set.
# Делаются предсказания модели на тестовых данных.
# Таким образом, в коде реализуется разбиение данных, обучение модели с оценкой качества на отложенной выборке и последующие предсказания на тесте.
cancer = load_breast_cancer()

X = cancer.data
y = cancer.target

xgb_model = xgb.XGBClassifier(objective="binary:logistic", n_estimators=20, random_state=42, eval_metric=["auc", "error", "error@0.6"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])

y_pred = xgb_model.predict(X_test)

[0]	validation_0-auc:0.96286	validation_0-error:0.06993	validation_0-error@0.6:0.06294
[1]	validation_0-auc:0.97742	validation_0-error:0.03497	validation_0-error@0.6:0.03497
[2]	validation_0-auc:0.97514	validation_0-error:0.04196	validation_0-error@0.6:0.04196
[3]	validation_0-auc:0.98096	validation_0-error:0.03497	validation_0-error@0.6:0.04196
[4]	validation_0-auc:0.98637	validation_0-error:0.03497	validation_0-error@0.6:0.04196
[5]	validation_0-auc:0.99043	validation_0-error:0.03497	validation_0-error@0.6:0.03497
[6]	validation_0-auc:0.98918	validation_0-error:0.03497	validation_0-error@0.6:0.03497
[7]	validation_0-auc:0.98980	validation_0-error:0.03497	validation_0-error@0.6:0.03497
[8]	validation_0-auc:0.98939	validation_0-error:0.03497	validation_0-error@0.6:0.03497
[9]	validation_0-auc:0.99251	validation_0-error:0.03497	validation_0-error@0.6:0.03497
[10]	validation_0-auc:0.99209	validation_0-error:0.03497	validation_0-error@0.6:0.04196
[11]	validation_0-auc:0.99168	validation_0

<h1>В ходе работы я последовательно решал задачи регрессии и классификации с использованием библиотеки xgboost.

Для регрессии были использованы данные о диабете. Модель XGBRegressor обучалась предсказывать уровень глюкозы. Была реализована кросс-валидация для оценки качества модели по метрике RMSE. Также проводился случайный перебор гиперпараметров для поиска наилучшей конфигурации модели. Это позволило найти оптимальные настройки и получить адекватную оценку качества модели.

Для классификации использовались данные о раке молочной железы и классификации вин. Были обучены модели XGBClassifier и оценено качество с помощью матрицы ошибок и метрики AUC. Также применялось разделение данных на обучение и тест для более объективной оценки модели.

Применение xgboost позволило эффективно решить задачи машинного обучения за счет использования градиентного бустинга. Данный подход хорошо себя зарекомендовал для работы с табличными данными. Полученные модели обладают хорошим качеством и могут быть применены на практике. </h1>